In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


# Adding necessary Libraries

In [2]:
import pandas as pd
from IPython.display import display

# Adding Titanic dataset

In [3]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')

# Checking Datasets

In [4]:
display(train_data.head())
display(test_data.head())


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


# Data Preprocessing

1. Check the number of null or missing values

In [5]:
print(train_data.isnull().sum())
print("Test: ",test_data.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
Test:  PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


As cabin has many missing values dropping it from both train and test data

In [6]:
train_data.drop(['Cabin'],axis=1,inplace=True)  # axis is added to tell that it is a column and not a row
test_data.drop(['Cabin'],axis=1,inplace=True)   # inplace=True modifies train_data directly without needing to assign it back to train_data

Taking the median and adding it in the missing places of Age

In [7]:
train_data['Age'].fillna(train_data['Age'].median(),inplace=True)
test_data['Age'].fillna(test_data['Age'].median(),inplace=True)

/tmp/ipykernel_17/600431010.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data['Age'].fillna(train_data['Age'].median(),inplace=True)
/tmp/ipykernel_17/600431010.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=T

Change all text columns to number values
One-hot encode the text columns using pandas get_dummies


In [8]:
train_data = pd.get_dummies(train_data,columns=['Sex','Embarked'], drop_first=True)
test_data = pd.get_dummies(test_data,columns=['Sex','Embarked'], drop_first=True)

In [9]:
# filling the missing value for Fare in test_data
test_data['Fare'].fillna(test_data['Fare'].median(),inplace=True)
print(f"Train: {len(train_data.columns)}\n Test: {len(test_data.columns)}")
# Train has Survived as one more column

Train: 12
 Test: 11


/tmp/ipykernel_17/3031759542.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_data['Fare'].fillna(test_data['Fare'].median(),inplace=True)


# Splitting Data

In [10]:
X = train_data.drop(['Name','Survived','Ticket','PassengerId'],axis=1)  # the independent variables or features we are going to use
y = train_data['Survived'] # the target variable we are trying to predict

X_test = test_data.drop(['Name','Ticket','PassengerId'],axis=1) # the test data of the X data

# Linear Regression Model

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score

X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.2,random_state=42)

model = LinearRegression()
model.fit(X_train,y_train)

y_pred = model.predict(X_val)
# Convert continuous predictions to binary (0 or 1)
y_pred_binary = (y_pred >= 0.5).astype(int)

# Evaluate using accuracy_score (classification metric)
print("Accuracy: ", accuracy_score(y_val, y_pred_binary))

Accuracy:  0.7932960893854749


Predictions

In [12]:
predictions = (model.predict(X_test) >= 0.5).astype(int)    # as linear regression is not a classsification model

# Saving to submission file
submission = pd.DataFrame({'PassengerId': test_data['PassengerId'],'Survived':predictions})

submission.to_csv('submission.csv',index=False)